In [1]:
import json
import os

import shutil
import base64
import requests
import pandas as pd 

from dotenv import load_dotenv
import google.generativeai as genai

In [2]:
# fdata = []
# data = []
# with open("dataset/chitron-captions-fresh.json", "r+") as f:
#     data = json.load(f)

In [182]:
# df = pd.json_normalize(data)
# df.loc[:,'Generated'] = None

df = pd.read_csv("dataset/generated_chitron.csv")

In [183]:
# df.to_csv("dummy.csv", index=False)
df.head()

,filename,caption,Generated,longest_caption
0,1.png,['তিন জন মেয়ে মানুষ আছে। এক জন দাড়িয়ে আছে আর দ...,"Q# ""ছবিতে মহিলাটির হাতে কি আছে?"", A# ""বেত""",'একটি হলুদ জামা পায়জামা পরা মহিলা দাড়িয়ে হাতে...
1,2.png,['জলাশয় এ একজন ছেলে মানুষ ও একটি বাচ্চা মানুষ ...,"Q# ""নৌকায় দাঁড়ানো পুরুষটি কি পরে আছেন?"", A# ""স...",'২ জন মানুষ যার একজন পুরুষ একটি নৌকায় দাড়িয়ে ...
2,3.png,"['অনেক মেয়ে মানুষ বসে আছে।', 'একটি নীল জামা পর...","Q# ""ল্যাপটপ ধারণকারী মহিলার জামার রঙ কি?"", A# ...",'একটি নীল জামা পরা মহিলা একটি নীল ল্যাপটপ এর ...
3,4.png,"['অনেক মানুষ একসাথে বসে কাজ করছে।', '২ টি ছোট...","Q# ""ছবিতে বসে থাকা ছোট ছেলেটি কী ধরনের পোশাক প...",'২ টি ছোট ছেলে একজন শার্ট প্যান্ট দাড়িয়ে চে...
4,5.png,"['ছয় জন মানুষ দাড়িয়ে আছে।', '৬ জন মানুষ এলোমেল...","Q# ""এই ছবিতে কতজন পুরুষ লুঙ্গী পরে আছেন?""\nA# ...","'৬ জন মানুষ এলোমেলো দাড়িয়ে আছে, তাদের মাঝে ২ ..."


In [184]:
# validation = df[mask]
df.isna().sum()

filename              0
caption               0
Generated          6234
longest_caption       0
dtype: int64

In [185]:

load_dotenv()

OPENAI_API = os.getenv("OPENAI_API")
GEMINI_API = os.getenv("GOOGLE_API")

In [186]:
# Import the Images module from pillow 
from PIL import Image 
  

def encode_image(image_path):
        
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')    

In [187]:
def prompt(captions):
    return """
    You are an expert Bengali question, answer generating assistant. You always generate the question and answer from \
    the given image and the captions to the image. you always have to abide by the conditions that are mentioned below: \n \n  
    CONDITION 1:  The questions are always image aligned and caption aligned and informative \n
    CONDITION 2:  Try to generate answer of one or two words. And the answer must never contain more than three words \n
    CONDITION 3:  Generate the question answer pair in Bengali language \n
        \n \n \
    Here is the caption:\n \n {},\
        \n \nBased on the above captions and below given image, generate ONE Question,Answer pair 
        based on this picture in Bengali language. generate the question answer pair like the following format:
    
    Q# "generated question", A# "generated answer"

    """.format(captions)

In [188]:
def give_q_n_a(image_name, captions):
    # Path to your image
    image_path = f"dataset/chitron/{image_name}"

    # Encode the image
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API}"
    }
    payload = {
        # "model": "gpt-4-vision-preview", "gpt-4-turbo"
        "model": "gpt-4-turbo",
        # "response_format": {"type": "json_object"},
        "messages": [
            {
                "role": "user",
                "content": [
                    # {"type": "text", "text": "Caption the image explaining the contents in it. Use Bengali language to caption the image."},
                    {"type": "text", "text": prompt(captions)},

                    {"type": "image_url", "image_url": {"url": 
    f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 100
    }   


    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    return response.json()#['choices'][0]['message']['content']


In [193]:
import time

sliced_df = df[2435:2500]

In [194]:
# total_tokens = 2520663 #2397670 #1393072 # 325355
# total_input_tokens = 2848053 #2288098 #2176137 #405867 # 293207
# total_output_tokens = 556623 #493624 # 482592 # 1537880 # 293207
#  36.23913 # 3.8965100000000006 $  #45.17922
val = 0

In [195]:
sliced_df.isna().sum()

filename            0
caption             0
Generated          65
longest_caption     0
dtype: int64

In [196]:
for idx, rows in sliced_df.iterrows():
    res = (give_q_n_a(rows['filename'], rows['longest_caption']))

    total_input_tokens = total_input_tokens + res['usage']['prompt_tokens']
    total_output_tokens = total_output_tokens + res['usage']['completion_tokens']
    total_tokens = total_tokens + res['usage']['total_tokens']
    
    df.loc[df['filename'] == str(rows['filename']), 'Generated'] = str(res['choices'][0]['message']['content'])
    
    print("done: idx: ",idx," filename: ", str(rows['filename']), "input_token: ", res['usage']['prompt_tokens'], "output_token: ", res['usage']['completion_tokens'])
    val = val+1
    time.sleep(.6)
    # break

done: idx:  2435  filename:  2930.png input_token:  499 output_token:  56
done: idx:  2436  filename:  2931.png input_token:  525 output_token:  55
done: idx:  2437  filename:  2932.png input_token:  514 output_token:  70
done: idx:  2438  filename:  2933.png input_token:  530 output_token:  70
done: idx:  2439  filename:  2934.png input_token:  536 output_token:  66
done: idx:  2440  filename:  2935.png input_token:  506 output_token:  67
done: idx:  2441  filename:  2936.png input_token:  510 output_token:  61
done: idx:  2442  filename:  2937.png input_token:  513 output_token:  62
done: idx:  2443  filename:  2938.png input_token:  558 output_token:  42
done: idx:  2444  filename:  2939.png input_token:  529 output_token:  70
done: idx:  2445  filename:  2940.png input_token:  518 output_token:  74
done: idx:  2446  filename:  2941.png input_token:  494 output_token:  50
done: idx:  2447  filename:  2942.png input_token:  549 output_token:  52
done: idx:  2448  filename:  2943.png 

In [198]:
# 3500- 4016
# res

In [192]:
sliced_df.head(10)

,filename,caption,Generated,longest_caption
2434,2929.png,"['অনেকগুলো মানুষ নামাজ পড়ছে।', 'অনেকগুলো পুরুষ...","Q# ""মানুষগুলো কোন কার্যক্রমে অংশ নিচ্ছে?"", A# ...",'অনেকগুলো পুরুষ মানুষ লাইন হয়ে বসে সিজদা দিয়ে...
2435,2930.png,"['দুইটি শিশু মাথার উপর ছাতা দিয়ে বসে আছে।', 'এ...",NaN,'একটি ভিজা জায়গায় উপরে ছাতা ধরে বসে আছে ২ জন ...
2436,2931.png,"['তিনটি কুকুর শুয়ে আছে।', 'একটি পাকার উপরে ঘুম...",NaN,'একটি পাকার উপরে ঘুমিয়ে আছে ৩ টি কুকুর এবং এক...
2437,2932.png,"['একসাথে অনেকগুলো নারী ও পুরুষ আছে।', 'অনেকগুল...",NaN,'অনেকগুলো নারী পুরুষ বাচ্চা একটি রুমে দাড়িয়ে ...
2438,2933.png,"['দুইজন নারী দাড়িয়ে আছে।', 'একটি জমির মাঝে শাড়...",NaN,'একটি জমির মাঝে শাড়ি পরা ২ জন মহিলা দাড়িয়ে আছ...
2439,2934.png,['অনেকগুলো পুরুষ বসে আছে। পিছনে অনেকগুলো মানুষ...,NaN,'একটি সমাবেশের প্যানেলে বসে আছে কিছু পুরুষ এব...
2440,2935.png,['কয়েকজন নারী শাড়ি পরে নৃত্য করছে। পিছনে কয়েকজ...,NaN,'কিছু মহিলা শাড়ি পরে একটি মঞ্চের মেঝেতে নাচ প...
2441,2936.png,"['চারজন পুরুষ আছে।', '৪ জন পুলিশ একটি রাস্তা দ...",NaN,'৪ জন পুলিশ একটি রাস্তা দিয়ে মারামারির পস্তুত...
2442,2937.png,"['মসজিদে অনেকগুলো মানুষ আছে।', 'একটি মসজিদে অন...",NaN,'একটি মসজিদে অনেকগুলো পুরুষ ছেলে বসে আছে এলো...
2443,2938.png,['একজন মানুষ ঘোড়ায় আছে। দূরে অনেকগুলো মানুষ আছ...,NaN,'একটি সাদা ঘোড়া দৌর দিচ্ছে একটি ছোট ছেলে কাঁদ...


In [179]:
print("total input tokens: ", total_input_tokens)
print("total output tokens: ", total_output_tokens)
print("total token count: " , total_tokens)
print(f"total cost: {total_input_tokens * 1e-5 + total_output_tokens * 3e-5}")

total input tokens:  3562644
total output tokens:  636898
total token count:  3315529
total cost: 54.733380000000004


In [180]:
df.isna().sum()

filename              0
caption               0
Generated          6234
longest_caption       0
dtype: int64

In [199]:
df.to_csv("dataset/generated_chitron.csv", index = False)

## Resizing the data 

In [119]:
import os

images = os.listdir("dataset/chitron_original")
len(images)

9154

In [126]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


for i in images:
    image = cv2.imread(f"dataset/chitron_original/{i}")
    try:
        resized = cv2.resize(image, (500, 500))
    except e:
        print(i, e)
    
    cv2.imwrite(f"dataset/chitron/{i}", resized)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

True